### Unit Tests of FFSR Module Functions ###

In [ ]:
# Date: 4/28-29/15

In [7]:
# rm -rf __pycache__/

In [1]:
%%file test_df_type.py

import numpy as np
import pandas as pd
from numpy.testing import assert_raises
from ffsr import df_type

def test_df_warn():
    x = list(np.arange(5))
    assert_raises(ValueError,df_type,x)
    
def test_df_df():
    x1 = np.arange(15).reshape(3,5)
    x2 = pd.DataFrame(x1)
    assert df_type(x1)*1+df_type(x2)*1 == 2

Overwriting test_df_type.py


In [4]:
%%file test_covnames.py

import readline
import ffsr as f
from numpy.testing import assert_raises

import rpy2.robjects as ro
import pandas.rpy.common as com
from rpy2.robjects.packages import importr

import numpy as np
import pandas as pd

leaps = importr('leaps')
stats = importr('stats')
base = importr('base')

regsub = ro.r('leaps::regsubsets')

np.random.seed(1234)

X = np.random.multivariate_normal(np.zeros(15),np.eye(15),(100))
beta = np.array([0,0,5,6,0,0,4,0,0,0,5,0,0,0,0]).reshape(15,1) # signif betas: 3,4,7,11
Y = X.dot(beta)
Y2 = pd.DataFrame(Y)
X2 = pd.DataFrame(X)
X2.columns = ["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15"]

fwd_r = f.forward(X2,Y2)

def test_nonames():    
    assert_raises(TypeError,f.cov_order,None)

Overwriting test_covnames.py


In [14]:
#%load_ext rpy2.ipython # <- doesn't help test_covnames.py run properly with py.test

In [6]:
%%file test_pvals.py

from ffsr import *

import readline
import rpy2.robjects as ro
import pandas.rpy.common as com

import numpy as np
import pandas as pd

np.random.seed(1234)

X = np.random.multivariate_normal(np.zeros(15),np.eye(15),(100))
beta = np.array([0,0,5,6,0,0,4,0,0,0,5,0,0,0,0]).reshape(15,1) # signif betas: 3,4,7,11
Y = X.dot(beta)
Y2 = pd.DataFrame(Y)
X2 = pd.DataFrame(X)
X2.columns = ["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15"]

fwd_r = forward(X2,Y2)
codnames = cov_order(X2.columns.values)

def test_probability():    
    assert 0. <= np.all(pval_comp(X2.shape[1])) <= 1

Overwriting test_pvals.py


In [1]:
%%file test_gamma.py

from ffsr import gamma_F
from numpy.testing import assert_raises

import numpy as np

np.random.seed(1234)

po = np.random.random(15)
pm = np.array([max(po[:(i+1)]) for i in range(len(po))])
s = np.array([len(np.where(pm<=pm[i])[0]) for i in range(len(pm))])

def test_nopvals():    
    assert_raises(TypeError,gamma_F,None,15,s)
    
def test_stoobig():
    assert_raises(ValueError,gamma_F,pm[:10],10,s)
    
def test_ncovtoosmall():
    assert_raises(ValueError,gamma_F,pm,10,s)

Overwriting test_gamma.py


In [2]:
%%file test_alpha.py

from ffsr import alpha_F
from numpy.testing import assert_raises, assert_almost_equal
import numpy as np

s = np.arange(1,15)

def test_nogamma0():    
    assert_raises(ValueError,alpha_F,None,15,s)
    
def test_noncov():
    assert_raises(ValueError,alpha_F,0.05,None,s)
    
def test_stoobig():
    assert_raises(ValueError,alpha_F,0.05,10,s)
    
def test_known():
    s = np.arange(1,5)
    n = 5
    a = 0.5 * (1 + s) / (n - s)
    af = alpha_F(0.5,n,np.arange(1,n))
    assert_almost_equal(a,af)

Overwriting test_alpha.py


In [12]:
%%file test_alphag.py

from ffsr import alpha_F_g
from numpy.testing import assert_raises, assert_almost_equal

import numpy as np

np.random.seed(1234)

g = np.random.random(15)
g2 = np.array([0.01,0.02,0.03])

def test_nogamma0():    
    assert_raises(ValueError,alpha_F_g,None,g,15)
    
def test_nogamma1():
    assert_raises(ValueError,alpha_F_g,0.05,None,15)
    
def test_nogamma2():
    assert_raises(ValueError,alpha_F_g,g2,None,15)
    
def test_g0toobig1():
    assert_raises(ValueError,alpha_F_g,1,g,15)
    
def test_g0toobig2():
    assert_raises(ValueError,alpha_F_g,np.array([0.05,1]),g,15)
    
def test_g0toosm1():
    assert_raises(ValueError,alpha_F_g,-0.1,g,15)
    
def test_g0toosm2():
    assert_raises(ValueError,alpha_F_g,np.array([-0.05,0.1]),g,15)
    
def test_g0toobgsm():
    assert_raises(ValueError,alpha_F_g,np.array([-0.05,0.1,1.]),g,15)
    
def test_noncov1():
    assert_raises(TypeError,alpha_F_g,0.05,g,None)
    
def test_noncov1():
    assert_raises(TypeError,alpha_F_g,g2,g,None)
    
def test_known():
    g3 = np.array([0.1,0.5])
    n = 5
    g0 = 0.25
    s = min(np.where(g3>g0)[0])
    a = 0.25*2/4. # g0 * (1. + s) / (n - s)
    afg = alpha_F_g(g0,g3,n)
    assert_almost_equal(a,afg)

Overwriting test_alphag.py


In [6]:
%%file test_beta.py

from ffsr import beta_est
from numpy.testing import assert_raises

import numpy as np
import pandas as pd

np.random.seed(1234)

g = np.random.random(15)

X = np.random.multivariate_normal(np.zeros(15),np.eye(15),(100))
beta = np.array([0,0,5,6,0,0,4,0,0,0,5,0,0,0,0]).reshape(15,1) # signif betas: 3,4,7,11
Y = X.dot(beta)
Y2 = pd.DataFrame(Y)
X2 = pd.DataFrame(X)
X2.columns = ["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15"]

def test_nox():
    assert_raises(AttributeError,beta_est,None,Y2,0.05,g,X2.columns.values)

def test_noy():
    assert_raises(TypeError,beta_est,X2,None,0.05,g,X2.columns.values)
    
def test_nog0():
    assert_raises(ValueError,beta_est,X2,Y2,None,g,X2.columns.values)
    
def test_nog():
    assert_raises(ValueError,beta_est,X2,Y2,0.05,None,X2.columns.values)

def test_nocovnames():
    assert_raises(TypeError,beta_est,X2,Y2,0.05,g,None)

Overwriting test_beta.py


In [24]:
%%file test_ffsr.py

from ffsr import ffsr
from numpy.testing import assert_raises

import numpy as np
import pandas as pd

np.random.seed(1234)

g = np.random.random(15)

X = np.random.multivariate_normal(np.zeros(15),np.eye(15),(100))
beta = np.array([0,0,5,6,0,0,4,0,0,0,5,0,0,0,0]).reshape(15,1) # signif betas: 3,4,7,11
Y = X.dot(beta)
Y2 = pd.DataFrame(Y)
X2 = pd.DataFrame(X)
X2.columns = ["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15"]
d = pd.concat([Y2,X2],axis=1)

def test_toomanyp():
    assert_raises(ValueError,ffsr,d.iloc[:10,],0.05)

def test_invalidg():
    assert_raises(ValueError,ffsr,d,0)

Writing test_ffsr.py


In [26]:
%%file test_bagfsr.py

from ffsr import bagfsr
from numpy.testing import assert_raises

import numpy as np
import pandas as pd

np.random.seed(1234)

g = np.random.random(15)

X = np.random.multivariate_normal(np.zeros(15),np.eye(15),(100))
beta = np.array([0,0,5,6,0,0,4,0,0,0,5,0,0,0,0]).reshape(15,1) # signif betas: 3,4,7,11
Y = X.dot(beta)
Y2 = pd.DataFrame(Y)
X2 = pd.DataFrame(X)
X2.columns = ["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15"]
d = pd.concat([Y2,X2],axis=1)

def test_toomanyp():
    assert_raises(ValueError,bagfsr,d.iloc[:10,],0.05)

def test_invalidg():
    assert_raises(ValueError,bagfsr,d,0)
    
def test_invalidB():
    assert_raises(ValueError,bagfsr,d,0.05,0)

Writing test_bagfsr.py


In [3]:
! py.test
# See bottom cells for expected results of test_covnames.py, test_pvals.py

============================= test session starts ==============================
platform linux2 -- Python 2.7.9 -- py-1.4.25 -- pytest-2.6.3
collected 31 items 

test_alpha.py ....
test_alphag.py ..........
test_bagfsr.py ...
test_beta.py .....
test_covnames.py .
test_df_type.py ..
test_ffsr.py ..
test_gamma.py ...
test_pvals.py .

========================== 31 passed in 2.62 seconds ===========================
